## Testing Method Module
---

In [21]:
import crowded.simulate as cs
import crowded.method as cm

#### Simulate the tasks

In [22]:
total_tasks = 415 
p_hard_tasks = 0.4
df_tasks = cs.Tasks(5).create(total_tasks, p_hard_tasks)
df_tasks.head()

,true_answers,label_task,prob_task
task_JDZnp3,cabs,hard_task,0.74
task_Dirbqy,cabs,easy_task,0.90
task_pTScyk,equator,hard_task,0.61
task_tbGB8W,validation,easy_task,0.90
task_uADBqS,church,easy_task,0.87


#### Simulate the workers

In [4]:
total_workers = 40 #50 #30
workers = cs.Workers().create(total_workers)
workers.head()

,prob_worker
c6bgrL,0.964732
J4WU3i,0.653160
waDsXx,0.734977
6af2m2,0.820328
nTDGFE,0.836270


#### Split the tasks

In [5]:
#Tasks Split
PTT = .3
def tasks_split(df, p = PTT):
    _train = df.sample(frac=p, random_state=23)
    _rest = df.loc[df.index.difference(_train.index)]
    return _train, _rest

In [6]:
tasks_train, tasks_rest = tasks_split(df_tasks, .3)

#### Assigning workers to tasks

In [7]:
wpt = 5
#workers per task always smaller than the number of workers
df_tw = cs.AssignTasks(tasks_train, workers, wpt).create()
df_tw.head()

,task_id,worker_id,true_answers,label_task,prob_task,prob_worker
0,task_w9KutQ,nveTHz,nozzle,easy_task,0.77,0.788873
1,task_w9KutQ,eWMRo3,nozzle,easy_task,0.77,0.944850
2,task_w9KutQ,4YsHNx,nozzle,easy_task,0.77,0.605178
3,task_w9KutQ,ryiFgi,nozzle,easy_task,0.77,0.411575
4,task_w9KutQ,yyNdV4,nozzle,easy_task,0.77,0.991780


#### Compute the probability to assess the tasks

In [8]:
keys = df_tasks['true_answers'].unique()
keys

array(['tax', 'nozzle', 'parks', 'instructions', 'conjecture'],
      dtype=object)

In [9]:
cp = cm.ComputeProbability(df_tw['prob_task'], df_tw['prob_worker'], keys)
df_tw['worker_answers'] = cm.WorkerAnswer(cp.predict(), df_tw['true_answers'], keys).match()
df_tw['performance'] = cp.predict()
df_tw.head()

,task_id,worker_id,true_answers,label_task,prob_task,prob_worker,worker_answers,performance
0,task_w9KutQ,nveTHz,nozzle,easy_task,0.77,0.788873,nozzle,1
1,task_w9KutQ,eWMRo3,nozzle,easy_task,0.77,0.944850,nozzle,1
2,task_w9KutQ,4YsHNx,nozzle,easy_task,0.77,0.605178,nozzle,1
3,task_w9KutQ,ryiFgi,nozzle,easy_task,0.77,0.411575,nozzle,1
4,task_w9KutQ,yyNdV4,nozzle,easy_task,0.77,0.991780,nozzle,1


#### Assess the performance and get the good workers

In [10]:
perf = cm.Performance(df_tw)
df_workers = perf._workers()
df_workers.head()

,prob_task,prob_worker,performance,worker_hability,tasks
worker_id,,,,,
44Qbis,0.701579,0.939656,1.0,good_worker,19
RHpT7M,0.754286,0.860358,1.0,good_worker,21
yyNdV4,0.758824,0.991780,1.0,good_worker,17
woW989,0.752667,0.980400,1.0,good_worker,15
v2pw3r,0.779375,0.784510,1.0,good_worker,16


In [11]:
good_workers = workers.loc[perf.good_workers()]
good_workers

,prob_worker
44Qbis,0.939656
RHpT7M,0.860358
woW989,0.980400
kDqi3j,0.956604
jsSqqT,0.944919
c6bgrL,0.964732
XPkAXg,0.926382
5ckety,0.994151
KSojAn,0.899101


#### Check in case there's undone tasks (no concensus achieved)

In [12]:
tasks = df_tw.groupby('task_id').mean().sort_values('performance', ascending=False)

In [13]:
undone = tasks[tasks['performance'] < 0.5]
if len(undone) > 0:
    #paste undone to tasks_rest
    print('There is {} tasks without concesus'.format(len(undone)))

#### Assign the good workers to the rest of the tasks

In [14]:
wpt = 5
#workers per task always smaller than the number of workers
df_tw_2 = cs.AssignTasks(tasks_rest, good_workers, wpt).create()
df_tw_2.head()

,task_id,worker_id,true_answers,label_task,prob_task,prob_worker
0,task_29exRM,kDqi3j,parks,hard_task,0.51,0.956604
1,task_29exRM,jsSqqT,parks,hard_task,0.51,0.944919
2,task_29exRM,c6bgrL,parks,hard_task,0.51,0.964732
3,task_29exRM,woW989,parks,hard_task,0.51,0.980400
4,task_29exRM,5ckety,parks,hard_task,0.51,0.994151


#### Compute probability to the rest of the tasks

In [15]:
cp2 = cm.ComputeProbability(df_tw_2['prob_task'], df_tw_2['prob_worker'], keys)
df_tw_2['worker_answers'] = cm.WorkerAnswer(cp2.predict(), df_tw_2['true_answers'], keys).match()
df_tw_2['performance'] = cp2.predict()
df_tw_2.head()

,task_id,worker_id,true_answers,label_task,prob_task,prob_worker,worker_answers,performance
0,task_29exRM,kDqi3j,parks,hard_task,0.51,0.956604,parks,1
1,task_29exRM,jsSqqT,parks,hard_task,0.51,0.944919,parks,1
2,task_29exRM,c6bgrL,parks,hard_task,0.51,0.964732,parks,1
3,task_29exRM,woW989,parks,hard_task,0.51,0.980400,parks,1
4,task_29exRM,5ckety,parks,hard_task,0.51,0.994151,parks,1


#### Merge the data and get the real accuracy

In [16]:
df = df_tw.append(df_tw_2)

In [20]:
from pycm import *
mat = ConfusionMatrix(df['true_answers'].tolist(), df['worker_answers'].tolist())
print(mat.Overall_ACC, mat.matrix())

Predict          conjectureinstructionsnozzle   parks    tax      
Actual
conjecture       419      0        2        0        4        
instructions     3        405      1        4        2        
nozzle           1        1        316      1        1        
parks            3        5        3        450      4        
tax              2        0        8        1        439      

0.9778313253012049 None


#### Compare with 1 stage

In [21]:
df_tw1 = cs.AssignTasks(df_tasks, workers, wpt).create()
cp1 = cm.ComputeProbability(df_tw1['prob_task'], df_tw1['prob_worker'], keys)
df_tw1['worker_answers'] = cm.WorkerAnswer(cp1.predict(), df_tw1['true_answers'], keys).match()
df_tw1['performance'] = cp1.predict()

In [22]:
mat = ConfusionMatrix(df_tw1['true_answers'].tolist(), df_tw1['worker_answers'].tolist())
print(mat.Overall_ACC, mat.matrix())

Predict          conjectureinstructionsnozzle   parks    tax      
Actual
conjecture       399      5        4        9        8        
instructions     9        389      8        4        5        
nozzle           5        6        303      5        1        
parks            8        5        6        440      6        
tax              5        11       9        7        418      

0.9392771084337349 None


#####  The accuracy of 1 stage algorithm is lower than the 2 stages algorithm *

---
### Overall Accuracy

In [13]:
total_tasks = 415 
total_workers = 40 
p_hard_tasks = 0.4
PTT = .3
wpt = 5
NK = 5

In [14]:
df = make.crowd_table(total_tasks, total_workers, p_hard_tasks, PTT, wpt, NK)
mat = ConfusionMatrix(df['true_answers'].tolist(), df['worker_answers'].tolist())
print(mat.Overall_ACC)

0.9763855421686747


In [1]:
import time, sys
sys.path.insert(0, '/Users/pedrohserrano/crowdED/crowded')
import simulate as cs
import method as cm
from pycm import *
import make
import pandas as pd

In [2]:
tasks = [60, 80, 100, 120, 140]
workers = [30, 40]
hard_t = [0.2, 0.4, 0.6, 0.8]
prop = [0.4, 0.6]
wpt = [3, 5, 7]
key = [3, 5, 7]

In [9]:
def _combinations(tasks, workers, hard_t, prop, wpt, key):
    table = []
    for t in tasks:
        for w in workers:
            for h in hard_t:
                for p in prop:
                    for x in wpt:
                        for k in key:
                            table.append([t, w, h, p, x, k])
    return table

def get_accuracy(tasks, workers, hard_t, prop, wpt, key):
    sim = _combinations(tasks, workers, hard_t, prop, wpt, key)
    for idx, l in enumerate(sim):
        make.update_progress("CrowdED simulation", idx/len(sim))
        try:
            df = make.crowd_table(total_tasks=l[0], total_workers=l[1], p_hard_tasks=l[2], PTT=l[3], wpt=l[4], NK=l[5])
            mat = ConfusionMatrix(df['true_answers'].tolist(), df['worker_answers'].tolist())
            l.insert(6, round(mat.Overall_ACC,4))
        except Exception:
            pass
    return sim

In [10]:
sim = get_accuracy(tasks, workers, hard_t, prop, wpt, key)
simulations = pd.DataFrame(sim)
simulations = simulations.fillna(0)
simulations.columns = ['total_tasks','total_workers','proportion_hard_tasks','proportion_train_tasks','workers_per_task','total_keys','accuracy']

CrowdED simulation: [####################] 99.86%

In [11]:
simulations[simulations['accuracy'] == 0]

,total_tasks,total_workers,proportion_hard_tasks,proportion_train_tasks,workers_per_task,total_keys,accuracy
177,80,30,0.4,0.6,7,3,0.0
303,100,30,0.2,0.6,7,3,0.0
322,100,30,0.4,0.6,7,5,0.0
345,100,30,0.8,0.4,5,3,0.0
349,100,30,0.8,0.4,7,5,0.0
359,100,30,0.8,0.6,7,7,0.0
447,120,30,0.2,0.6,7,3,0.0
462,120,30,0.4,0.6,5,3,0.0
465,120,30,0.4,0.6,7,3,0.0
466,120,30,0.4,0.6,7,5,0.0


In [13]:
simulations.to_csv('../data/df_crowded.csv', index=False)